# Script de comparaison des textes de PF par similarité cosine

## t-SNE

In [3]:
#!/usr/bin/env python3
"""
Script d'analyse de similarité des discours politiques Tours 1+2 combinés
Avec anonymisation complète : suppression des noms, prénoms et partis
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import networkx as nx
import plotly.graph_objects as go
import warnings
import re

warnings.filterwarnings('ignore')

class CombinedToursAnalyzer:
    def __init__(self, filepath='/Users/charlielezin/Desktop/Candidatures58-81-250825.csv'):
        self.filepath = filepath
        self.df = None
        self.similarity_matrix = None
        self.vectorizer = None
        self.embeddings_2d = None
        
    def load_and_filter_data(self):
        """
        Charge les données CSV et filtre pour les tours 1+2
        """
        print("Chargement des données (Tours 1+2)...")
        self.df = pd.read_csv(self.filepath, delimiter=';', encoding='utf-8')
        
        print(f"Nombre total de lignes : {len(self.df)}")
        
        # Filtrage pour les tours 1 et 2
        self.df = self.df[self.df['tour'].isin([1, 2])].copy()
        print(f"Filtrage tours 1+2 : {len(self.df)} candidatures")
        
        # Filtrer ceux qui ont des discours non vides
        self.df = self.df[
            (self.df['discours'].notna()) & 
            (self.df['discours'].str.strip() != '') &
            (self.df['discours'].str.len() > 100)
        ].copy()
        
        print(f"Nombre de candidatures avec discours : {len(self.df)}")
        self.df = self.df.reset_index(drop=True)
        
        print(f"Répartition par tour :")
        print(self.df['tour'].value_counts().sort_index())
        
        print(f"Répartition par nuance politique :")
        nuances_count = self.df['nuance_titulaire'].fillna('Inconnu').value_counts()
        print(nuances_count.head(10))
        
        return self.df
    
    def extract_names_and_parties(self):
        """
        Extrait tous les noms, prénoms et mentions de partis à supprimer
        """
        names_to_remove = set()
        parties_to_remove = set()
        
        for _, row in self.df.iterrows():
            # Noms et prénoms titulaire et suppléant
            for col in ['prenom_titulaire', 'nom_titulaire', 'prenom_suppleant', 'nom_suppleant']:
                if pd.notna(row.get(col)):
                    names_to_remove.add(str(row[col]).strip().lower())
            
            # Partis et sigles
            party_columns = [
                'parti_titulaire', 'parti_titulaire_1', 'parti_titulaire_2', 
                'parti_titulaire_3', 'parti_titulaire_4',
                'parti_suppleant', 'parti_suppleant_1', 'parti_suppleant_2',
                'parti_suppleant_3', 'parti_suppleant_4', 'parti_suppleant_5',
                'sigle_titulaire', 'sigle_suppleant', 'nuance_titulaire', 'nuance_suppleant'
            ]
            
            for col in party_columns:
                if col in row and pd.notna(row[col]):
                    party_value = str(row[col]).strip()
                    if party_value and party_value.lower() not in ['nan', 'none', '']:
                        party_parts = re.split(r'[+\-\s&/]', party_value)
                        for part in party_parts:
                            if part.strip() and len(part.strip()) > 1:
                                parties_to_remove.add(part.strip().lower())
        
        # Termes de parti communs
        common_parties = [
            'gaulliste', 'gaullistes', 'socialiste', 'socialistes', 'communiste', 'communistes',
            'radical', 'radicaux', 'centriste', 'centristes', 'républicain', 'républicains',
            'démocrate', 'démocrates', 'indépendant', 'indépendants', 'paysan', 'paysans',
            'union', 'mouvement', 'parti', 'rassemblement', 'front', 'coalition',
            'nouvelle', 'république', 'nationale', 'populaire', 'française', 'français',
            'gaulle', 'gaulles', 'gaul', 'com', 'soc', 'sfio', 'mrp', 'cnip', 'unr',
            'psu', 'agr', 'exg', 'exd', 'div', 'rdg', 'rad', 'cen'
        ]
        
        parties_to_remove.update(common_parties)
        
        # Nettoyer les sets
        names_to_remove = {name for name in names_to_remove if name and len(name) > 1}
        parties_to_remove = {party for party in parties_to_remove if party and len(party) > 2}
        
        print(f"Anonymisation:")
        print(f"   Noms/prénoms à supprimer: {len(names_to_remove)}")
        print(f"   Termes de partis à supprimer: {len(parties_to_remove)}")
        
        return names_to_remove, parties_to_remove
    
    def anonymize_text(self, text, names_to_remove, parties_to_remove):
        """
        Anonymise un texte en supprimant les noms et mentions de partis
        """
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        
        # Supprimer les noms et prénoms
        for name in names_to_remove:
            text = re.sub(rf'\b{re.escape(name)}\b', ' ', text, flags=re.IGNORECASE)
        
        # Supprimer les mentions de partis
        for party in parties_to_remove:
            text = re.sub(rf'\b{re.escape(party)}\b', ' ', text, flags=re.IGNORECASE)
        
        # Nettoyage général
        text = re.sub(r'\n+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\d+', '', text)
        text = text.strip()
        
        return text
    
    def get_french_stop_words(self):
        """
        Liste étendue de mots vides français
        """
        stop_words_base = [
            'le', 'de', 'et', 'à', 'un', 'il', 'être', 'en', 'avoir', 'que', 'pour',
            'dans', 'ce', 'son', 'une', 'sur', 'avec', 'ne', 'se', 'pas', 'tout', 'plus',
            'par', 'grand', 'comme', 'mais', 'faire', 'mettre', 'nom', 'dire', 'ces',
            'mon', 'où', 'même', 'y', 'aller', 'deux', 'moi', 'si', 'haut', 'bien', 'autre',
            'fois', 'très', 'là', 'voir', 'arriver', 'donner', 'elle', 'lui', 'tel', 'quel',
            'qui', 'du', 'des', 'aux', 'cette', 'leurs', 'nos', 'votre', 'dont', 'cette'
        ]
        
        stop_words_political = [
            'vous', 'nous', 'électeurs', 'électrices', 'france', 'français', 'française',
            'république', 'national', 'nationale', 'gouvernement', 'politique', 'pays',
            'état', 'public', 'sociale', 'social', 'économique', 'candidat', 'candidats',
            'voter', 'vote', 'élection', 'élections', 'législatives', 'député', 'assemblée',
            'monsieur', 'madame', 'citoyens', 'citoyennes', 'peuple', 'nation',
            'suppléant', 'titulaire', 'liste', 'scrutin', 'circonscription'
        ]
        
        return stop_words_base + stop_words_political
    
    def create_color_palette(self, nuances):
        """
        Palette de couleurs pour les nuances politiques
        """
        color_map = {
            'COM': '#FF0000', 'COMM': '#FF0000', 'GAUL': '#191970', 'SOC': '#FFB6C1',
            'CEN': '#87CEEB', 'EXG': '#8B0000', 'RAD': '#DAA520', 'PSU': '#FF8C00',
            'AGR': '#008000', 'CEN-D': '#0000FF', 'EXD': '#000000', 'DIV': '#808080',
            'RDG': '#FFE4E1', 'SFIO': '#FFB6C1', 'MRP': '#FF8C00', 'UDSR': '#87CEEB',
            'RGR': '#FFE4E1', 'DROITE': '#191970', 'DROIT': '#191970', 'IND': '#808080',
            'PAYSANS': '#008000'
        }
        
        # Gérer les nuances manquantes
        unique_nuances = list(set(nuances))
        remaining_nuances = [n for n in unique_nuances if n not in color_map and pd.notna(n)]
        
        if remaining_nuances:
            additional_colors = ['#DDA0DD', '#F0E68C', '#98FB98', '#F5DEB3', 
                               '#D2B48C', '#BC8F8F', '#CD853F', '#A0522D']
            for i, nuance in enumerate(remaining_nuances):
                color_map[nuance] = additional_colors[i % len(additional_colors)]
        
        color_map[np.nan] = '#CCCCCC'
        color_map['Inconnu'] = '#CCCCCC'
        color_map[None] = '#CCCCCC'
        
        return color_map
    
    def calculate_tfidf_similarity(self):
        """
        Calcule la matrice TF-IDF et la similarité sur les textes anonymisés
        """
        print("Anonymisation des textes...")
        
        # Extraire les noms et partis à supprimer
        names_to_remove, parties_to_remove = self.extract_names_and_parties()
        
        # Anonymiser tous les discours
        self.df['discours_anonymized'] = self.df['discours'].apply(
            lambda x: self.anonymize_text(x, names_to_remove, parties_to_remove)
        )
        
        # Filtrer les textes trop courts après anonymisation
        self.df = self.df[self.df['discours_anonymized'].str.len() > 50].copy()
        self.df = self.df.reset_index(drop=True)
        
        print(f"Nombre de discours après anonymisation : {len(self.df)}")
        
        if len(self.df) < 5:
            raise ValueError("Pas assez de discours après anonymisation!")
        
        print("Calcul de la matrice TF-IDF sur textes anonymisés...")
        
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            min_df=2,
            max_df=0.8,
            stop_words=self.get_french_stop_words(),
            ngram_range=(1, 2),
            lowercase=True,
            strip_accents='unicode'
        )
        
        texts = self.df['discours_anonymized'].tolist()
        tfidf_matrix = self.vectorizer.fit_transform(texts)
        
        print(f"Matrice TF-IDF : {tfidf_matrix.shape}")
        
        print("Calcul de la similarité cosinus...")
        self.similarity_matrix = cosine_similarity(tfidf_matrix)
        
        # Statistiques
        similarity_upper = np.triu(self.similarity_matrix, k=1)
        non_zero_similarities = similarity_upper[similarity_upper > 0]
        
        print(f"Statistiques de similarité (textes anonymisés) :")
        print(f"  - Moyenne : {non_zero_similarities.mean():.3f}")
        print(f"  - Médiane : {np.median(non_zero_similarities):.3f}")
        print(f"  - Max : {non_zero_similarities.max():.3f}")
        print(f"  - Min : {non_zero_similarities.min():.3f}")
        
        # Calcul du t-SNE
        print("Calcul du t-SNE...")
        tsne = TSNE(n_components=2, perplexity=min(30, len(self.df)-1), 
                   random_state=42, verbose=0, max_iter=1000)
        self.embeddings_2d = tsne.fit_transform(tfidf_matrix.toarray())
        
        return tfidf_matrix
    
    def analyze_similar_groups(self, top_n=50):
        """
        Analyse des groupes de discours similaires
        """
        print(f"TOP {top_n} ASSOCIATIONS SIMILAIRES (TEXTES ANONYMISÉS)")
        print("="*60)
        
        high_similarity_threshold = 0.6
        
        G = nx.Graph()
        for i in range(len(self.df)):
            G.add_node(i)
        
        similarity_pairs = []
        for i in range(len(self.similarity_matrix)):
            for j in range(i+1, len(self.similarity_matrix)):
                similarity = self.similarity_matrix[i, j]
                if similarity >= high_similarity_threshold:
                    G.add_edge(i, j, weight=similarity)
                    similarity_pairs.append((similarity, i, j))
        
        similarity_pairs.sort(reverse=True)
        
        connected_components = list(nx.connected_components(G))
        groups = [comp for comp in connected_components if len(comp) > 2]
        
        print(f"Analyse avec seuil >= {high_similarity_threshold}")
        print(f"Trouvé: {len(groups)} groupes, {len(similarity_pairs)} paires similaires")
        
        associations_count = 0
        
        # Afficher les groupes
        if groups and associations_count < top_n:
            print("GROUPES DE DISCOURS SIMILAIRES (>=3 documents):")
            print("-" * 50)
            
            for group in groups:
                if associations_count >= top_n:
                    break
                
                group_list = list(group)
                group_size = len(group_list)
                
                print(f"\n{associations_count + 1}. GROUPE DE {group_size} DOCUMENTS SIMILAIRES:")
                
                # Similarité moyenne interne
                internal_similarities = []
                for i in range(len(group_list)):
                    for j in range(i+1, len(group_list)):
                        internal_similarities.append(self.similarity_matrix[group_list[i], group_list[j]])
                
                avg_similarity = np.mean(internal_similarities)
                print(f"   Similarité moyenne interne: {avg_similarity:.3f}")
                
                # Afficher les membres
                for idx, doc_idx in enumerate(group_list):
                    row_data = self.df.iloc[doc_idx]
                    nom = f"{row_data['prenom_titulaire']} {row_data['nom_titulaire']}"
                    nuance = row_data['nuance_titulaire']
                    annee = int(row_data['annee_scrutin']) if pd.notna(row_data['annee_scrutin']) else 'N/A'
                    tour = row_data.get('tour', 'N/A')
                    
                    print(f"   - {nom} ({nuance}, {annee}, T{tour})")
                
                associations_count += 1
        
        # Afficher les paires
        if associations_count < top_n:
            print(f"\nPAIRES DE DOCUMENTS TRÈS SIMILAIRES:")
            print("-" * 50)
            
            for similarity, i, j in similarity_pairs:
                if associations_count >= top_n:
                    break
                
                # Vérifier que pas déjà dans un groupe
                already_in_group = any(i in group or j in group for group in groups)
                if already_in_group:
                    continue
                
                row_data_i = self.df.iloc[i]
                row_data_j = self.df.iloc[j]
                
                nom_i = f"{row_data_i['prenom_titulaire']} {row_data_i['nom_titulaire']}"
                nuance_i = row_data_i['nuance_titulaire']
                
                nom_j = f"{row_data_j['prenom_titulaire']} {row_data_j['nom_titulaire']}"
                nuance_j = row_data_j['nuance_titulaire']
                
                print(f"\n{associations_count + 1}. Similarité: {similarity:.3f}")
                print(f"   - {nom_i} ({nuance_i})")
                print(f"   - {nom_j} ({nuance_j})")
                
                associations_count += 1
        
        print(f"\nRÉSUMÉ (ANALYSE ANONYMISÉE):")
        print(f"   Associations affichées: {associations_count}")
        print(f"   Groupes identifiés: {len(groups)}")
        
        return groups, similarity_pairs
    
    def analyze_cross_tour_similarities(self):
        """
        Analyse spéciale des similarités croisées entre tours
        """
        print(f"SIMILARITÉS CROISÉES TOUR 1 <-> TOUR 2:")
        print("-" * 50)
        
        tours = self.df['tour']
        tour1_indices = np.where(tours == 1)[0]
        tour2_indices = np.where(tours == 2)[0]
        
        if len(tour1_indices) == 0 or len(tour2_indices) == 0:
            print("Pas assez de données pour l'analyse croisée")
            return
        
        print(f"Tour 1: {len(tour1_indices)} discours")
        print(f"Tour 2: {len(tour2_indices)} discours")
        
        # Similarité inter-tours
        inter_similarities = []
        cross_tour_pairs = []
        for i in tour1_indices:
            for j in tour2_indices:
                similarity = self.similarity_matrix[i, j]
                inter_similarities.append(similarity)
                if similarity > 0.3:
                    cross_tour_pairs.append((similarity, i, j))
        
        avg_inter = np.mean(inter_similarities)
        print(f"Similarité moyenne inter-tours: {avg_inter:.3f}")
        
        cross_tour_pairs.sort(reverse=True)
        
        print(f"Top 10 similarités entre Tour 1 et Tour 2:")
        for idx, (similarity, i, j) in enumerate(cross_tour_pairs[:10], 1):
            row_i = self.df.iloc[i]
            row_j = self.df.iloc[j]
            
            nom_i = f"{row_i['prenom_titulaire']} {row_i['nom_titulaire']}"
            nuance_i = row_i['nuance_titulaire']
            
            nom_j = f"{row_j['prenom_titulaire']} {row_j['nom_titulaire']}"
            nuance_j = row_j['nuance_titulaire']
            
            print(f"{idx:2d}. Similarité: {similarity:.3f}")
            print(f"    T1: {nom_i} ({nuance_i})")
            print(f"    T2: {nom_j} ({nuance_j})")
    
    def create_interactive_tsne(self):
        """
        Visualisation t-SNE interactive
        """
        print("Création de la visualisation t-SNE interactive...")
        
        nuances = self.df['nuance_titulaire'].fillna('Inconnu')
        color_palette = self.create_color_palette(nuances)
        
        hover_texts = []
        for _, row in self.df.iterrows():
            nom = f"{row['prenom_titulaire']} {row['nom_titulaire']}"
            nuance = row['nuance_titulaire'] if pd.notna(row['nuance_titulaire']) else 'Inconnu'
            annee = int(row['annee_scrutin']) if pd.notna(row['annee_scrutin']) else 'N/A'
            tour = row.get('tour', 'N/A')
            departement = row.get('departement_nom', 'N/A')
            circonscription = row.get('identifiant_circonscription', 'N/A')
            
            hover_text = f"""
            <b>{nom}</b><br>
            <b>Nuance:</b> {nuance}<br>
            <b>Année:</b> {annee}<br>
            <b>Tour:</b> {tour}<br>
            <b>Département:</b> {departement}<br>
            <b>Circonscription:</b> {circonscription}<br>
            <i>Analyse sur texte anonymisé</i>
            """
            hover_texts.append(hover_text)
        
        fig = go.Figure()
        
        for nuance in set(nuances):
            mask = nuances == nuance
            if sum(mask) > 0:
                indices = np.where(mask)[0]
                
                fig.add_trace(go.Scatter(
                    x=self.embeddings_2d[mask, 0],
                    y=self.embeddings_2d[mask, 1],
                    mode='markers',
                    name=f'{nuance} (n={sum(mask)})',
                    marker=dict(
                        color=color_palette.get(nuance, '#CCCCCC'),
                        size=8,
                        line=dict(width=1, color='black'),
                        opacity=0.8
                    ),
                    hovertemplate='%{customdata}<extra></extra>',
                    customdata=[hover_texts[i] for i in indices],
                ))
        
        fig.update_layout(
            title='Analyse t-SNE Interactive - Tours 1+2 (Textes Anonymisés)<br><sub>Noms, prénoms et partis supprimés de l\'analyse</sub>',
            xaxis_title='Dimension t-SNE 1',
            yaxis_title='Dimension t-SNE 2',
            width=1200,
            height=800,
            hovermode='closest',
            showlegend=True,
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.01)
        )
        
        return fig

def main():
    """
    Analyse principale Tours 1+2 avec anonymisation
    """
    print("ANALYSE TF-IDF TOURS 1+2 COMBINÉS - TEXTES ANONYMISÉS")
    print("="*60)
    
    try:
        analyzer = CombinedToursAnalyzer()
        
        # 1. Chargement des données
        analyzer.load_and_filter_data()
        
        if len(analyzer.df) == 0:
            print("Aucun discours trouvé!")
            return
        
        # 2. Calcul TF-IDF avec anonymisation
        analyzer.calculate_tfidf_similarity()
        
        # 3. Analyse des groupes similaires
        analyzer.analyze_similar_groups(top_n=50)
        
        # 4. Analyse croisée entre tours
        analyzer.analyze_cross_tour_similarities()
        
        # 5. Visualisation interactive
        fig_tsne = analyzer.create_interactive_tsne()
        fig_tsne.write_html("tsne_tours_combined_anonymized.html")
        
        print(f"Visualisation sauvegardée: tsne_tours_combined_anonymized.html")
        print("Analyse terminée")
        
    except Exception as e:
        print(f"Erreur : {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

ANALYSE TF-IDF TOURS 1+2 COMBINÉS - TEXTES ANONYMISÉS
Chargement des données (Tours 1+2)...
Nombre total de lignes : 303
Filtrage tours 1+2 : 303 candidatures
Nombre de candidatures avec discours : 303
Répartition par tour :
tour
1    217
2     86
Name: count, dtype: int64
Répartition par nuance politique :
nuance_titulaire
GAUL     62
SOC      58
COM      54
CEN      36
RAD      21
EXG      15
CEN-D    15
PSU      11
AGR      11
EXD      10
Name: count, dtype: int64
Anonymisation des textes...
Anonymisation:
   Noms/prénoms à supprimer: 370
   Termes de partis à supprimer: 135
Nombre de discours après anonymisation : 303
Calcul de la matrice TF-IDF sur textes anonymisés...
Matrice TF-IDF : (303, 1000)
Calcul de la similarité cosinus...
Statistiques de similarité (textes anonymisés) :
  - Moyenne : 0.165
  - Médiane : 0.156
  - Max : 1.000
  - Min : 0.003
Calcul du t-SNE...
TOP 50 ASSOCIATIONS SIMILAIRES (TEXTES ANONYMISÉS)
Analyse avec seuil >= 0.6
Trouvé: 7 groupes, 63 paires similai

In [6]:
#!/usr/bin/env python3
"""
Script de clustering pour l'analyse des discours politiques Tours 1+2
Basé sur le code d'analyse existant avec ajout du clustering et visualisation des clusters
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import networkx as nx
import plotly.graph_objects as go
from plotly.colors import qualitative
import warnings
import re

warnings.filterwarnings('ignore')

class ClusteringToursAnalyzer:
    def __init__(self, filepath='/Users/charlielezin/Desktop/Candidatures58-81-250825.csv'):
        self.filepath = filepath
        self.df = None
        self.similarity_matrix = None
        self.vectorizer = None
        self.embeddings_2d = None
        self.tfidf_matrix = None
        self.clusters = None
        self.n_clusters = None
        
    def load_and_filter_data(self):
        """
        Charge les données CSV et filtre pour les tours 1+2
        """
        print("Chargement des données (Tours 1+2)...")
        self.df = pd.read_csv(self.filepath, delimiter=';', encoding='utf-8')
        
        print(f"Nombre total de lignes : {len(self.df)}")
        
        # Filtrage pour les tours 1 et 2
        self.df = self.df[self.df['tour'].isin([1, 2])].copy()
        print(f"Filtrage tours 1+2 : {len(self.df)} candidatures")
        
        # Filtrer ceux qui ont des discours non vides
        self.df = self.df[
            (self.df['discours'].notna()) & 
            (self.df['discours'].str.strip() != '') &
            (self.df['discours'].str.len() > 100)
        ].copy()
        
        print(f"Nombre de candidatures avec discours : {len(self.df)}")
        self.df = self.df.reset_index(drop=True)
        
        print(f"Répartition par tour :")
        print(self.df['tour'].value_counts().sort_index())
        
        print(f"Répartition par nuance politique :")
        nuances_count = self.df['nuance_titulaire'].fillna('Inconnu').value_counts()
        print(nuances_count.head(10))
        
        return self.df
    
    def extract_names_and_parties(self):
        """
        Extrait tous les noms, prénoms et mentions de partis à supprimer
        """
        names_to_remove = set()
        parties_to_remove = set()
        
        for _, row in self.df.iterrows():
            # Noms et prénoms titulaire et suppléant
            for col in ['prenom_titulaire', 'nom_titulaire', 'prenom_suppleant', 'nom_suppleant']:
                if pd.notna(row.get(col)):
                    names_to_remove.add(str(row[col]).strip().lower())
            
            # Partis et sigles
            party_columns = [
                'parti_titulaire', 'parti_titulaire_1', 'parti_titulaire_2', 
                'parti_titulaire_3', 'parti_titulaire_4',
                'parti_suppleant', 'parti_suppleant_1', 'parti_suppleant_2',
                'parti_suppleant_3', 'parti_suppleant_4', 'parti_suppleant_5',
                'sigle_titulaire', 'sigle_suppleant', 'nuance_titulaire', 'nuance_suppleant'
            ]
            
            for col in party_columns:
                if col in row and pd.notna(row[col]):
                    party_value = str(row[col]).strip()
                    if party_value and party_value.lower() not in ['nan', 'none', '']:
                        party_parts = re.split(r'[+\-\s&/]', party_value)
                        for part in party_parts:
                            if part.strip() and len(part.strip()) > 1:
                                parties_to_remove.add(part.strip().lower())
        
        # Termes de parti communs
        common_parties = [
            'gaulliste', 'gaullistes', 'socialiste', 'socialistes', 'communiste', 'communistes',
            'radical', 'radicaux', 'centriste', 'centristes', 'républicain', 'républicains',
            'démocrate', 'démocrates', 'indépendant', 'indépendants', 'paysan', 'paysans',
            'union', 'mouvement', 'parti', 'rassemblement', 'front', 'coalition',
            'nouvelle', 'république', 'nationale', 'populaire', 'française', 'français',
            'gaulle', 'gaulles', 'gaul', 'com', 'soc', 'sfio', 'mrp', 'cnip', 'unr',
            'psu', 'agr', 'exg', 'exd', 'div', 'rdg', 'rad', 'cen'
        ]
        
        parties_to_remove.update(common_parties)
        
        # Nettoyer les sets
        names_to_remove = {name for name in names_to_remove if name and len(name) > 1}
        parties_to_remove = {party for party in parties_to_remove if party and len(party) > 2}
        
        print(f"Anonymisation:")
        print(f"   Noms/prénoms à supprimer: {len(names_to_remove)}")
        print(f"   Termes de partis à supprimer: {len(parties_to_remove)}")
        
        return names_to_remove, parties_to_remove
    
    def anonymize_text(self, text, names_to_remove, parties_to_remove):
        """
        Anonymise un texte en supprimant les noms et mentions de partis
        """
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        
        # Supprimer les noms et prénoms
        for name in names_to_remove:
            text = re.sub(rf'\b{re.escape(name)}\b', ' ', text, flags=re.IGNORECASE)
        
        # Supprimer les mentions de partis
        for party in parties_to_remove:
            text = re.sub(rf'\b{re.escape(party)}\b', ' ', text, flags=re.IGNORECASE)
        
        # Nettoyage général
        text = re.sub(r'\n+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\d+', '', text)
        text = text.strip()
        
        return text
    
    def get_french_stop_words(self):
        """
        Liste étendue de mots vides français
        """
        stop_words_base = [
            'le', 'de', 'et', 'à', 'un', 'il', 'être', 'en', 'avoir', 'que', 'pour',
            'dans', 'ce', 'son', 'une', 'sur', 'avec', 'ne', 'se', 'pas', 'tout', 'plus',
            'par', 'grand', 'comme', 'mais', 'faire', 'mettre', 'nom', 'dire', 'ces',
            'mon', 'où', 'même', 'y', 'aller', 'deux', 'moi', 'si', 'haut', 'bien', 'autre',
            'fois', 'très', 'là', 'voir', 'arriver', 'donner', 'elle', 'lui', 'tel', 'quel',
            'qui', 'du', 'des', 'aux', 'cette', 'leurs', 'nos', 'votre', 'dont', 'cette'
        ]
        
        stop_words_political = [
            'vous', 'nous', 'électeurs', 'électrices', 'france', 'français', 'française',
            'république', 'national', 'nationale', 'gouvernement', 'politique', 'pays',
            'état', 'public', 'sociale', 'social', 'économique', 'candidat', 'candidats',
            'voter', 'vote', 'élection', 'élections', 'législatives', 'député', 'assemblée',
            'monsieur', 'madame', 'citoyens', 'citoyennes', 'peuple', 'nation',
            'suppléant', 'titulaire', 'liste', 'scrutin', 'circonscription'
        ]
        
        return stop_words_base + stop_words_political
    
    def create_color_palette(self, nuances):
        """
        Palette de couleurs pour les nuances politiques
        """
        color_map = {
            'COM': '#FF0000', 'COMM': '#FF0000', 'GAUL': '#191970', 'SOC': '#FFB6C1',
            'CEN': '#87CEEB', 'EXG': '#8B0000', 'RAD': '#DAA520', 'PSU': '#FF8C00',
            'AGR': '#008000', 'CEN-D': '#0000FF', 'EXD': '#000000', 'DIV': '#808080',
            'RDG': '#FFE4E1', 'SFIO': '#FFB6C1', 'MRP': '#FF8C00', 'UDSR': '#87CEEB',
            'RGR': '#FFE4E1', 'DROITE': '#191970', 'DROIT': '#191970', 'IND': '#808080',
            'PAYSANS': '#008000'
        }
        
        # Gérer les nuances manquantes
        unique_nuances = list(set(nuances))
        remaining_nuances = [n for n in unique_nuances if n not in color_map and pd.notna(n)]
        
        if remaining_nuances:
            additional_colors = ['#DDA0DD', '#F0E68C', '#98FB98', '#F5DEB3', 
                               '#D2B48C', '#BC8F8F', '#CD853F', '#A0522D']
            for i, nuance in enumerate(remaining_nuances):
                color_map[nuance] = additional_colors[i % len(additional_colors)]
        
        color_map[np.nan] = '#CCCCCC'
        color_map['Inconnu'] = '#CCCCCC'
        color_map[None] = '#CCCCCC'
        
        return color_map
    
    def calculate_tfidf_similarity(self):
        """
        Calcule la matrice TF-IDF et la similarité sur les textes anonymisés
        """
        print("Anonymisation des textes...")
        
        # Extraire les noms et partis à supprimer
        names_to_remove, parties_to_remove = self.extract_names_and_parties()
        
        # Anonymiser tous les discours
        self.df['discours_anonymized'] = self.df['discours'].apply(
            lambda x: self.anonymize_text(x, names_to_remove, parties_to_remove)
        )
        
        # Filtrer les textes trop courts après anonymisation
        self.df = self.df[self.df['discours_anonymized'].str.len() > 50].copy()
        self.df = self.df.reset_index(drop=True)
        
        print(f"Nombre de discours après anonymisation : {len(self.df)}")
        
        if len(self.df) < 5:
            raise ValueError("Pas assez de discours après anonymisation!")
        
        print("Calcul de la matrice TF-IDF sur textes anonymisés...")
        
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            min_df=2,
            max_df=0.8,
            stop_words=self.get_french_stop_words(),
            ngram_range=(1, 2),
            lowercase=True,
            strip_accents='unicode'
        )
        
        texts = self.df['discours_anonymized'].tolist()
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)
        
        print(f"Matrice TF-IDF : {self.tfidf_matrix.shape}")
        
        print("Calcul de la similarité cosinus...")
        self.similarity_matrix = cosine_similarity(self.tfidf_matrix)
        
        # Statistiques
        similarity_upper = np.triu(self.similarity_matrix, k=1)
        non_zero_similarities = similarity_upper[similarity_upper > 0]
        
        print(f"Statistiques de similarité (textes anonymisés) :")
        print(f"  - Moyenne : {non_zero_similarities.mean():.3f}")
        print(f"  - Médiane : {np.median(non_zero_similarities):.3f}")
        print(f"  - Max : {non_zero_similarities.max():.3f}")
        print(f"  - Min : {non_zero_similarities.min():.3f}")
        
        # Calcul du t-SNE
        print("Calcul du t-SNE...")
        tsne = TSNE(n_components=2, perplexity=min(30, len(self.df)-1), 
                   random_state=42, verbose=0, max_iter=1000)
        self.embeddings_2d = tsne.fit_transform(self.tfidf_matrix.toarray())
        
        return self.tfidf_matrix
    
    def find_optimal_clusters(self, max_clusters=None):
        """Détermine le nombre optimal de clusters"""
        print("Recherche du nombre optimal de clusters...")
        
        if max_clusters is None:
            max_clusters = min(10, len(self.df) // 3)
        
        silhouette_scores = []
        
        for k in range(2, max_clusters + 1):
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            cluster_labels = kmeans.fit_predict(self.tfidf_matrix.toarray())
            
            silhouette_avg = silhouette_score(self.tfidf_matrix.toarray(), cluster_labels)
            silhouette_scores.append((k, silhouette_avg))
            
            print(f"  k={k}: Silhouette={silhouette_avg:.3f}")
        
        # Choisir le k avec le meilleur score de silhouette
        best_k = max(silhouette_scores, key=lambda x: x[1])[0]
        best_score = max(silhouette_scores, key=lambda x: x[1])[1]
        
        print(f"Nombre optimal de clusters : {best_k} (Silhouette: {best_score:.3f})")
        
        return best_k
    
    def perform_clustering(self, n_clusters=None):
        """Effectue le clustering"""
        if n_clusters is None:
            n_clusters = self.find_optimal_clusters()
        
        print(f"Clustering avec {n_clusters} clusters...")
        
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        self.clusters = kmeans.fit_predict(self.tfidf_matrix.toarray())
        self.n_clusters = n_clusters
        
        # Ajouter les clusters au DataFrame
        self.df['cluster'] = self.clusters
        
        print("Répartition des clusters :")
        for i in range(n_clusters):
            cluster_size = np.sum(self.clusters == i)
            print(f"  Cluster {i}: {cluster_size} documents")
        
        return self.clusters
    
    def analyze_clusters(self):
        """Analyse détaillée des clusters"""
        print(f"ANALYSE DES {self.n_clusters} CLUSTERS")
        print("="*50)
        
        for cluster_id in range(self.n_clusters):
            cluster_docs = self.df[self.df['cluster'] == cluster_id]
            
            print(f"\nCLUSTER {cluster_id} ({len(cluster_docs)} documents)")
            print("-" * 30)
            
            # Répartition par nuance
            nuances = cluster_docs['nuance_titulaire'].fillna('Inconnu').value_counts()
            print(f"Nuances principales:")
            for nuance, count in nuances.head(3).items():
                pct = (count / len(cluster_docs)) * 100
                print(f"  - {nuance}: {count} ({pct:.1f}%)")
            
            # Répartition par tour
            tours = cluster_docs['tour'].value_counts()
            print(f"Tours:")
            for tour, count in tours.items():
                pct = (count / len(cluster_docs)) * 100
                print(f"  - Tour {tour}: {count} ({pct:.1f}%)")
            
            # Exemples de candidats
            print("Exemples:")
            for _, row in cluster_docs.head(3).iterrows():
                nom = f"{row['prenom_titulaire']} {row['nom_titulaire']}"
                nuance = row['nuance_titulaire'] if pd.notna(row['nuance_titulaire']) else 'Inconnu'
                tour = row.get('tour', 'N/A')
                annee = int(row['annee_scrutin']) if pd.notna(row['annee_scrutin']) else 'N/A'
                print(f"  - {nom} ({nuance}, {annee}, T{tour})")
    
    def get_cluster_hull_points(self, cluster_id):
        """Calcule l'enveloppe des points d'un cluster"""
        cluster_mask = self.clusters == cluster_id
        if np.sum(cluster_mask) < 3:
            return None
        
        cluster_points = self.embeddings_2d[cluster_mask]
        
        try:
            from scipy.spatial import ConvexHull
            hull = ConvexHull(cluster_points)
            hull_points = cluster_points[hull.vertices]
            hull_points = np.vstack([hull_points, hull_points[0]])
            return hull_points
        except:
            # Utiliser un cercle approximatif si ConvexHull échoue
            center = np.mean(cluster_points, axis=0)
            distances = np.linalg.norm(cluster_points - center, axis=1)
            radius = np.max(distances) * 1.2
            
            angles = np.linspace(0, 2*np.pi, 30)
            circle_points = np.column_stack([
                center[0] + radius * np.cos(angles),
                center[1] + radius * np.sin(angles)
            ])
            return circle_points
    
    def create_interactive_tsne_with_clusters(self):
        """
        Visualisation t-SNE interactive avec clusters entourés
        """
        print("Création de la visualisation t-SNE avec clusters...")
        
        nuances = self.df['nuance_titulaire'].fillna('Inconnu')
        color_palette = self.create_color_palette(nuances)
        
        # Couleurs pour les enveloppes de clusters
        cluster_colors = qualitative.Plotly[:self.n_clusters]
        
        hover_texts = []
        for _, row in self.df.iterrows():
            nom = f"{row['prenom_titulaire']} {row['nom_titulaire']}"
            nuance = row['nuance_titulaire'] if pd.notna(row['nuance_titulaire']) else 'Inconnu'
            annee = int(row['annee_scrutin']) if pd.notna(row['annee_scrutin']) else 'N/A'
            tour = row.get('tour', 'N/A')
            cluster_id = row.get('cluster', 'N/A')
            departement = row.get('departement_nom', 'N/A')
            circonscription = row.get('identifiant_circonscription', 'N/A')
            
            hover_text = (f"<b>{nom}</b><br>"
                         f"<b>Nuance:</b> {nuance}<br>"
                         f"<b>Cluster:</b> {cluster_id}<br>"
                         f"<b>Année:</b> {annee}<br>"
                         f"<b>Tour:</b> {tour}<br>"
                         f"<b>Département:</b> {departement}<br>"
                         f"<b>Circonscription:</b> {circonscription}<br>"
                         f"<i>Analyse sur texte anonymisé</i>")
            hover_texts.append(hover_text)
        
        fig = go.Figure()
        
        # Ajouter d'abord les enveloppes des clusters
        for cluster_id in range(self.n_clusters):
            hull_points = self.get_cluster_hull_points(cluster_id)
            if hull_points is not None:
                cluster_size = np.sum(self.clusters == cluster_id)
                color_hex = cluster_colors[cluster_id]
                
                # Convertir la couleur hex en RGB pour le fill
                r = int(color_hex[1:3], 16)
                g = int(color_hex[3:5], 16)
                b = int(color_hex[5:7], 16)
                
                fig.add_trace(go.Scatter(
                    x=hull_points[:, 0],
                    y=hull_points[:, 1],
                    mode='lines',
                    line=dict(color=color_hex, width=2, dash='dash'),
                    name=f'Cluster {cluster_id} ({cluster_size} docs)',
                    fill='toself',
                    fillcolor=f'rgba({r}, {g}, {b}, 0.1)',
                    opacity=0.6,
                    showlegend=True
                ))
        
        # Ajouter ensuite les points par nuance politique
        for nuance in set(nuances):
            mask = nuances == nuance
            if sum(mask) > 0:
                indices = np.where(mask)[0]
                
                fig.add_trace(go.Scatter(
                    x=self.embeddings_2d[mask, 0],
                    y=self.embeddings_2d[mask, 1],
                    mode='markers',
                    name=f'{nuance} (n={sum(mask)})',
                    marker=dict(
                        color=color_palette.get(nuance, '#CCCCCC'),
                        size=8,
                        line=dict(width=1, color='black'),
                        opacity=0.8
                    ),
                    hovertemplate='%{customdata}<extra></extra>',
                    customdata=[hover_texts[i] for i in indices]
                ))
        
        fig.update_layout(
            title=f'Analyse t-SNE avec {self.n_clusters} Clusters - Tours 1+2<br><sub>Clusters entourés sur textes anonymisés</sub>',
            xaxis_title='Dimension t-SNE 1',
            yaxis_title='Dimension t-SNE 2',
            width=1400,
            height=900,
            hovermode='closest',
            showlegend=True,
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.01),
            plot_bgcolor='white'
        )
        
        return fig

def main():
    """
    Analyse principale avec clustering
    """
    print("ANALYSE DE CLUSTERING - DISCOURS POLITIQUES TOURS 1+2")
    print("="*60)
    
    try:
        analyzer = ClusteringToursAnalyzer()
        
        # 1. Chargement des données
        analyzer.load_and_filter_data()
        
        if len(analyzer.df) == 0:
            print("Aucun discours trouvé!")
            return
        
        # 2. Calcul TF-IDF avec anonymisation
        analyzer.calculate_tfidf_similarity()
        
        # 3. Clustering
        analyzer.perform_clustering()
        
        # 4. Analyse des clusters
        analyzer.analyze_clusters()
        
        # 5. Visualisation interactive avec clusters entourés
        fig = analyzer.create_interactive_tsne_with_clusters()
        fig.write_html("clustering_tours_combined.html")
        
        print(f"Visualisation avec clusters sauvegardée: clustering_tours_combined.html")
        print("Analyse de clustering terminée")
        
    except Exception as e:
        print(f"Erreur : {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

ANALYSE DE CLUSTERING - DISCOURS POLITIQUES TOURS 1+2
Chargement des données (Tours 1+2)...
Nombre total de lignes : 303
Filtrage tours 1+2 : 303 candidatures
Nombre de candidatures avec discours : 303
Répartition par tour :
tour
1    217
2     86
Name: count, dtype: int64
Répartition par nuance politique :
nuance_titulaire
GAUL     62
SOC      58
COM      54
CEN      36
RAD      21
EXG      15
CEN-D    15
PSU      11
AGR      11
EXD      10
Name: count, dtype: int64
Anonymisation des textes...
Anonymisation:
   Noms/prénoms à supprimer: 370
   Termes de partis à supprimer: 135
Nombre de discours après anonymisation : 303
Calcul de la matrice TF-IDF sur textes anonymisés...
Matrice TF-IDF : (303, 1000)
Calcul de la similarité cosinus...
Statistiques de similarité (textes anonymisés) :
  - Moyenne : 0.165
  - Médiane : 0.156
  - Max : 1.000
  - Min : 0.003
Calcul du t-SNE...
Recherche du nombre optimal de clusters...
  k=2: Silhouette=0.035
  k=3: Silhouette=0.028
  k=4: Silhouette=0.02

## Analyse avec DBSCAN directement sur la matrice de similarité

In [9]:
#!/usr/bin/env python3
"""
Script de clustering pour l'analyse des discours politiques Tours 1+2
Basé sur le code d'analyse existant avec ajout du clustering et visualisation des clusters
"""

warnings.filterwarnings('ignore')

class ClusteringToursAnalyzer:
    def __init__(self, filepath='/Users/charlielezin/Desktop/Candidatures58-81-250825.csv'):
        self.filepath = filepath
        self.df = None
        self.similarity_matrix = None
        self.vectorizer = None
        self.embeddings_2d = None
        self.tfidf_matrix = None
        self.clusters = None
        self.n_clusters = None
        
    def load_and_filter_data(self):
        """
        Charge les données CSV et filtre pour les tours 1+2
        """
        print("Chargement des données (Tours 1+2)...")
        self.df = pd.read_csv(self.filepath, delimiter=';', encoding='utf-8')
        
        print(f"Nombre total de lignes : {len(self.df)}")
        
        # Filtrage pour les tours 1 et 2
        self.df = self.df[self.df['tour'].isin([1, 2])].copy()
        print(f"Filtrage tours 1+2 : {len(self.df)} candidatures")
        
        # Filtrer ceux qui ont des discours non vides
        self.df = self.df[
            (self.df['discours'].notna()) & 
            (self.df['discours'].str.strip() != '') &
            (self.df['discours'].str.len() > 100)
        ].copy()
        
        print(f"Nombre de candidatures avec discours : {len(self.df)}")
        self.df = self.df.reset_index(drop=True)
        
        print(f"Répartition par tour :")
        print(self.df['tour'].value_counts().sort_index())
        
        print(f"Répartition par nuance politique :")
        nuances_count = self.df['nuance_titulaire'].fillna('Inconnu').value_counts()
        print(nuances_count.head(10))
        
        return self.df
    
    def extract_names_and_parties(self):
        """
        Extrait tous les noms, prénoms et mentions de partis à supprimer
        """
        names_to_remove = set()
        parties_to_remove = set()
        
        for _, row in self.df.iterrows():
            # Noms et prénoms titulaire et suppléant
            for col in ['prenom_titulaire', 'nom_titulaire', 'prenom_suppleant', 'nom_suppleant']:
                if pd.notna(row.get(col)):
                    names_to_remove.add(str(row[col]).strip().lower())
            
            # Partis et sigles
            party_columns = [
                'parti_titulaire', 'parti_titulaire_1', 'parti_titulaire_2', 
                'parti_titulaire_3', 'parti_titulaire_4',
                'parti_suppleant', 'parti_suppleant_1', 'parti_suppleant_2',
                'parti_suppleant_3', 'parti_suppleant_4', 'parti_suppleant_5',
                'sigle_titulaire', 'sigle_suppleant', 'nuance_titulaire', 'nuance_suppleant'
            ]
            
            for col in party_columns:
                if col in row and pd.notna(row[col]):
                    party_value = str(row[col]).strip()
                    if party_value and party_value.lower() not in ['nan', 'none', '']:
                        party_parts = re.split(r'[+\-\s&/]', party_value)
                        for part in party_parts:
                            if part.strip() and len(part.strip()) > 1:
                                parties_to_remove.add(part.strip().lower())
        
        # Termes de parti communs
        common_parties = [
            'gaulliste', 'gaullistes', 'socialiste', 'socialistes', 'communiste', 'communistes',
            'radical', 'radicaux', 'centriste', 'centristes', 'républicain', 'républicains',
            'démocrate', 'démocrates', 'indépendant', 'indépendants', 'paysan', 'paysans',
            'union', 'mouvement', 'parti', 'rassemblement', 'front', 'coalition',
            'nouvelle', 'république', 'nationale', 'populaire', 'française', 'français',
            'gaulle', 'gaulles', 'gaul', 'com', 'soc', 'sfio', 'mrp', 'cnip', 'unr',
            'psu', 'agr', 'exg', 'exd', 'div', 'rdg', 'rad', 'cen'
        ]
        
        parties_to_remove.update(common_parties)
        
        # Nettoyer les sets
        names_to_remove = {name for name in names_to_remove if name and len(name) > 1}
        parties_to_remove = {party for party in parties_to_remove if party and len(party) > 2}
        
        print(f"Anonymisation:")
        print(f"   Noms/prénoms à supprimer: {len(names_to_remove)}")
        print(f"   Termes de partis à supprimer: {len(parties_to_remove)}")
        
        return names_to_remove, parties_to_remove
    
    def anonymize_text(self, text, names_to_remove, parties_to_remove):
        """
        Anonymise un texte en supprimant les noms et mentions de partis
        """
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        
        # Supprimer les noms et prénoms
        for name in names_to_remove:
            text = re.sub(rf'\b{re.escape(name)}\b', ' ', text, flags=re.IGNORECASE)
        
        # Supprimer les mentions de partis
        for party in parties_to_remove:
            text = re.sub(rf'\b{re.escape(party)}\b', ' ', text, flags=re.IGNORECASE)
        
        # Nettoyage général
        text = re.sub(r'\n+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\d+', '', text)
        text = text.strip()
        
        return text
    
    def get_french_stop_words(self):
        """
        Liste étendue de mots vides français
        """
        stop_words_base = [
            'le', 'de', 'et', 'à', 'un', 'il', 'être', 'en', 'avoir', 'que', 'pour',
            'dans', 'ce', 'son', 'une', 'sur', 'avec', 'ne', 'se', 'pas', 'tout', 'plus',
            'par', 'grand', 'comme', 'mais', 'faire', 'mettre', 'nom', 'dire', 'ces',
            'mon', 'où', 'même', 'y', 'aller', 'deux', 'moi', 'si', 'haut', 'bien', 'autre',
            'fois', 'très', 'là', 'voir', 'arriver', 'donner', 'elle', 'lui', 'tel', 'quel',
            'qui', 'du', 'des', 'aux', 'cette', 'leurs', 'nos', 'votre', 'dont', 'cette'
        ]
        
        stop_words_political = [
            'vous', 'nous', 'électeurs', 'électrices', 'france', 'français', 'française',
            'république', 'national', 'nationale', 'gouvernement', 'politique', 'pays',
            'état', 'public', 'sociale', 'social', 'économique', 'candidat', 'candidats',
            'voter', 'vote', 'élection', 'élections', 'législatives', 'député', 'assemblée',
            'monsieur', 'madame', 'citoyens', 'citoyennes', 'peuple', 'nation',
            'suppléant', 'titulaire', 'liste', 'scrutin', 'circonscription'
        ]
        
        return stop_words_base + stop_words_political
    
    def create_color_palette(self, nuances):
        """
        Palette de couleurs pour les nuances politiques
        """
        color_map = {
            'COM': '#FF0000', 'COMM': '#FF0000', 'GAUL': '#191970', 'SOC': '#FFB6C1',
            'CEN': '#87CEEB', 'EXG': '#8B0000', 'RAD': '#DAA520', 'PSU': '#FF8C00',
            'AGR': '#008000', 'CEN-D': '#0000FF', 'EXD': '#000000', 'DIV': '#808080',
            'RDG': '#FFE4E1', 'SFIO': '#FFB6C1', 'MRP': '#FF8C00', 'UDSR': '#87CEEB',
            'RGR': '#FFE4E1', 'DROITE': '#191970', 'DROIT': '#191970', 'IND': '#808080',
            'PAYSANS': '#008000'
        }
        
        # Gérer les nuances manquantes
        unique_nuances = list(set(nuances))
        remaining_nuances = [n for n in unique_nuances if n not in color_map and pd.notna(n)]
        
        if remaining_nuances:
            additional_colors = ['#DDA0DD', '#F0E68C', '#98FB98', '#F5DEB3', 
                               '#D2B48C', '#BC8F8F', '#CD853F', '#A0522D']
            for i, nuance in enumerate(remaining_nuances):
                color_map[nuance] = additional_colors[i % len(additional_colors)]
        
        color_map[np.nan] = '#CCCCCC'
        color_map['Inconnu'] = '#CCCCCC'
        color_map[None] = '#CCCCCC'
        
        return color_map
    
    def calculate_tfidf_similarity(self):
        """
        Calcule la matrice TF-IDF et la similarité sur les textes anonymisés
        """
        print("Anonymisation des textes...")
        
        # Extraire les noms et partis à supprimer
        names_to_remove, parties_to_remove = self.extract_names_and_parties()
        
        # Anonymiser tous les discours
        self.df['discours_anonymized'] = self.df['discours'].apply(
            lambda x: self.anonymize_text(x, names_to_remove, parties_to_remove)
        )
        
        # Filtrer les textes trop courts après anonymisation
        self.df = self.df[self.df['discours_anonymized'].str.len() > 50].copy()
        self.df = self.df.reset_index(drop=True)
        
        print(f"Nombre de discours après anonymisation : {len(self.df)}")
        
        if len(self.df) < 5:
            raise ValueError("Pas assez de discours après anonymisation!")
        
        print("Calcul de la matrice TF-IDF sur textes anonymisés...")
        
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            min_df=2,
            max_df=0.8,
            stop_words=self.get_french_stop_words(),
            ngram_range=(1, 2),
            lowercase=True,
            strip_accents='unicode'
        )
        
        texts = self.df['discours_anonymized'].tolist()
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)
        
        print(f"Matrice TF-IDF : {self.tfidf_matrix.shape}")
        
        print("Calcul de la similarité cosinus...")
        self.similarity_matrix = cosine_similarity(self.tfidf_matrix)
        
        # Statistiques
        similarity_upper = np.triu(self.similarity_matrix, k=1)
        non_zero_similarities = similarity_upper[similarity_upper > 0]
        
        print(f"Statistiques de similarité (textes anonymisés) :")
        print(f"  - Moyenne : {non_zero_similarities.mean():.3f}")
        print(f"  - Médiane : {np.median(non_zero_similarities):.3f}")
        print(f"  - Max : {non_zero_similarities.max():.3f}")
        print(f"  - Min : {non_zero_similarities.min():.3f}")
        
        # Calcul du t-SNE
        print("Calcul du t-SNE...")
        tsne = TSNE(n_components=2, perplexity=min(30, len(self.df)-1), 
                   random_state=42, verbose=0, max_iter=1000)
        self.embeddings_2d = tsne.fit_transform(self.tfidf_matrix.toarray())
        
        return self.tfidf_matrix
    
    def find_optimal_clusters_dbscan(self):
        """Utilise DBSCAN pour identifier les clusters denses"""
        print("Clustering DBSCAN sur l'espace TF-IDF haute dimension...")
        
        # Tester différents paramètres eps pour DBSCAN
        eps_values = [0.3, 0.4, 0.5, 0.6, 0.7]
        best_eps = None
        best_n_clusters = 0
        best_noise_ratio = 1.0
        
        for eps in eps_values:
            dbscan = DBSCAN(eps=eps, min_samples=5)
            cluster_labels = dbscan.fit_predict(self.tfidf_matrix.toarray())
            
            n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
            noise_ratio = np.sum(cluster_labels == -1) / len(cluster_labels)
            
            print(f"  eps={eps}: {n_clusters} clusters, {noise_ratio:.1%} bruit")
            
            # Choisir la configuration avec un bon équilibre
            if n_clusters >= 2 and n_clusters <= 8 and noise_ratio < 0.5:
                if best_eps is None or (n_clusters > best_n_clusters and noise_ratio < best_noise_ratio):
                    best_eps = eps
                    best_n_clusters = n_clusters
                    best_noise_ratio = noise_ratio
        
        if best_eps is None:
            print("DBSCAN n'a pas trouvé de bonne configuration, utilisation de K-means avec k=4")
            return 4, "kmeans"
        
        print(f"Meilleur paramètre DBSCAN : eps={best_eps} ({best_n_clusters} clusters)")
        return best_eps, "dbscan"
    
    def perform_clustering(self, n_clusters=None):
        """Effectue le clustering sur l'espace TF-IDF haute dimension"""
        
        # Déterminer la méthode et paramètres optimaux
        param, method = self.find_optimal_clusters_dbscan()
        
        if method == "dbscan":
            print(f"Clustering DBSCAN avec eps={param}...")
            dbscan = DBSCAN(eps=param, min_samples=5)
            cluster_labels = dbscan.fit_predict(self.tfidf_matrix.toarray())
            
            # Traiter le bruit (-1) comme un cluster séparé
            self.clusters = cluster_labels
            unique_clusters = set(cluster_labels)
            self.n_clusters = len(unique_clusters)
            
        else:  # kmeans fallback
            print(f"Clustering K-means avec {param} clusters...")
            kmeans = KMeans(n_clusters=param, random_state=42, n_init=10)
            self.clusters = kmeans.fit_predict(self.tfidf_matrix.toarray())
            self.n_clusters = param
        
        # Ajouter les clusters au DataFrame
        self.df['cluster'] = self.clusters
        
        print("Répartition des clusters :")
        unique_labels = set(self.clusters)
        for label in sorted(unique_labels):
            cluster_size = np.sum(self.clusters == label)
            if label == -1:
                print(f"  Bruit: {cluster_size} documents")
            else:
                print(f"  Cluster {label}: {cluster_size} documents")
        
        return self.clusters
    
    def analyze_clusters(self):
        """Analyse détaillée des clusters"""
        print(f"ANALYSE DES CLUSTERS")
        print("="*50)
        
        unique_labels = set(self.clusters)
        for label in sorted(unique_labels):
            if label == -1:
                cluster_docs = self.df[self.df['cluster'] == -1]
                print(f"\nBRUIT ({len(cluster_docs)} documents)")
                print("-" * 30)
            else:
                cluster_docs = self.df[self.df['cluster'] == label]
                print(f"\nCLUSTER {label} ({len(cluster_docs)} documents)")
                print("-" * 30)
            
            # Répartition par nuance
            nuances = cluster_docs['nuance_titulaire'].fillna('Inconnu').value_counts()
            print(f"Nuances principales:")
            for nuance, count in nuances.head(3).items():
                pct = (count / len(cluster_docs)) * 100
                print(f"  - {nuance}: {count} ({pct:.1f}%)")
            
            # Répartition par tour
            tours = cluster_docs['tour'].value_counts()
            print(f"Tours:")
            for tour, count in tours.items():
                pct = (count / len(cluster_docs)) * 100
                print(f"  - Tour {tour}: {count} ({pct:.1f}%)")
            
            # Exemples de candidats
            print("Exemples:")
            for _, row in cluster_docs.head(3).iterrows():
                nom = f"{row['prenom_titulaire']} {row['nom_titulaire']}"
                nuance = row['nuance_titulaire'] if pd.notna(row['nuance_titulaire']) else 'Inconnu'
                tour = row.get('tour', 'N/A')
                annee = int(row['annee_scrutin']) if pd.notna(row['annee_scrutin']) else 'N/A'
                print(f"  - {nom} ({nuance}, {annee}, T{tour})")
    
    def get_cluster_hull_points(self, cluster_id):
        """Calcule l'enveloppe des points d'un cluster"""
        if cluster_id == -1:
            # Pour le bruit, pas d'enveloppe
            return None
            
        cluster_mask = self.clusters == cluster_id
        if np.sum(cluster_mask) < 3:
            return None
        
        cluster_points = self.embeddings_2d[cluster_mask]
        
        try:
            from scipy.spatial import ConvexHull
            hull = ConvexHull(cluster_points)
            hull_points = cluster_points[hull.vertices]
            hull_points = np.vstack([hull_points, hull_points[0]])
            return hull_points
        except:
            # Utiliser un cercle approximatif si ConvexHull échoue
            center = np.mean(cluster_points, axis=0)
            distances = np.linalg.norm(cluster_points - center, axis=1)
            radius = np.max(distances) * 1.2
            
            angles = np.linspace(0, 2*np.pi, 30)
            circle_points = np.column_stack([
                center[0] + radius * np.cos(angles),
                center[1] + radius * np.sin(angles)
            ])
            return circle_points
    
    def create_interactive_tsne_with_clusters(self):
        """
        Visualisation t-SNE interactive avec clusters entourés
        """
        print("Création de la visualisation t-SNE avec clusters...")
        
        nuances = self.df['nuance_titulaire'].fillna('Inconnu')
        color_palette = self.create_color_palette(nuances)
        
        # Couleurs pour les enveloppes de clusters
        cluster_colors = qualitative.Plotly[:self.n_clusters]
        
        hover_texts = []
        for _, row in self.df.iterrows():
            nom = f"{row['prenom_titulaire']} {row['nom_titulaire']}"
            nuance = row['nuance_titulaire'] if pd.notna(row['nuance_titulaire']) else 'Inconnu'
            annee = int(row['annee_scrutin']) if pd.notna(row['annee_scrutin']) else 'N/A'
            tour = row.get('tour', 'N/A')
            cluster_id = row.get('cluster', 'N/A')
            departement = row.get('departement_nom', 'N/A')
            circonscription = row.get('identifiant_circonscription', 'N/A')
            
            hover_text = f"<b>{nom}</b><br><b>Nuance:</b> {nuance}<br><b>Cluster:</b> {cluster_id}<br><b>Année:</b> {annee}<br><b>Tour:</b> {tour}<br><b>Département:</b> {departement}<br><b>Circonscription:</b> {circonscription}<br><i>Analyse sur texte anonymisé</i>"
            hover_texts.append(hover_text)
        
        fig = go.Figure()
        
        # Ajouter d'abord les enveloppes des clusters valides
        unique_labels = set(self.clusters)
        cluster_colors = qualitative.Plotly[:len(unique_labels)]
        color_idx = 0
        
        for cluster_id in sorted(unique_labels):
            if cluster_id != -1:  # Ignorer le bruit pour les enveloppes
                hull_points = self.get_cluster_hull_points(cluster_id)
                if hull_points is not None:
                    cluster_size = np.sum(self.clusters == cluster_id)
                    color_hex = cluster_colors[color_idx]
                    
                    # Convertir la couleur hex en RGB pour le fill
                    r = int(color_hex[1:3], 16)
                    g = int(color_hex[3:5], 16)
                    b = int(color_hex[5:7], 16)
                    
                    fig.add_trace(go.Scatter(
                        x=hull_points[:, 0],
                        y=hull_points[:, 1],
                        mode='lines',
                        line=dict(color=color_hex, width=2, dash='dash'),
                        name=f'Cluster {cluster_id} ({cluster_size} docs)',
                        fill='toself',
                        fillcolor=f'rgba({r}, {g}, {b}, 0.1)',
                        opacity=0.6,
                        showlegend=True
                    ))
                color_idx += 1
        
        # Ajouter ensuite les points par nuance politique
        for nuance in set(nuances):
            mask = nuances == nuance
            if sum(mask) > 0:
                indices = np.where(mask)[0]
                
                fig.add_trace(go.Scatter(
                    x=self.embeddings_2d[mask, 0],
                    y=self.embeddings_2d[mask, 1],
                    mode='markers',
                    name=f'{nuance} (n={sum(mask)})',
                    marker=dict(
                        color=color_palette.get(nuance, '#CCCCCC'),
                        size=8,
                        line=dict(width=1, color='black'),
                        opacity=0.8
                    ),
                    hovertemplate='%{customdata}<extra></extra>',
                    customdata=[hover_texts[i] for i in indices]
                ))
        
        fig.update_layout(
            title=f'Analyse t-SNE avec {self.n_clusters} Clusters - Tours 1+2<br><sub>Clusters entourés sur textes anonymisés</sub>',
            xaxis_title='Dimension t-SNE 1',
            yaxis_title='Dimension t-SNE 2',
            width=1400,
            height=900,
            hovermode='closest',
            showlegend=True,
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.01),
            plot_bgcolor='white'
        )
        
        return fig

def main():
    """
    Analyse principale avec clustering
    """
    print("ANALYSE DE CLUSTERING - DISCOURS POLITIQUES TOURS 1+2")
    print("="*60)
    
    try:
        analyzer = ClusteringToursAnalyzer()
        
        # 1. Chargement des données
        analyzer.load_and_filter_data()
        
        if len(analyzer.df) == 0:
            print("Aucun discours trouvé!")
            return
        
        # 2. Calcul TF-IDF avec anonymisation
        analyzer.calculate_tfidf_similarity()
        
        # 3. Clustering
        analyzer.perform_clustering()
        
        # 4. Analyse des clusters
        analyzer.analyze_clusters()
        
        # 5. Visualisation interactive avec clusters entourés
        fig = analyzer.create_interactive_tsne_with_clusters()
        fig.write_html("clustering_tours_combined.html")
        
        print(f"Visualisation avec clusters sauvegardée: clustering_tours_combined.html")
        print("Analyse de clustering terminée")
        
    except Exception as e:
        print(f"Erreur : {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

ANALYSE DE CLUSTERING - DISCOURS POLITIQUES TOURS 1+2
Chargement des données (Tours 1+2)...
Nombre total de lignes : 303
Filtrage tours 1+2 : 303 candidatures
Nombre de candidatures avec discours : 303
Répartition par tour :
tour
1    217
2     86
Name: count, dtype: int64
Répartition par nuance politique :
nuance_titulaire
GAUL     62
SOC      58
COM      54
CEN      36
RAD      21
EXG      15
CEN-D    15
PSU      11
AGR      11
EXD      10
Name: count, dtype: int64
Anonymisation des textes...
Anonymisation:
   Noms/prénoms à supprimer: 370
   Termes de partis à supprimer: 135
Nombre de discours après anonymisation : 303
Calcul de la matrice TF-IDF sur textes anonymisés...
Matrice TF-IDF : (303, 1000)
Calcul de la similarité cosinus...
Statistiques de similarité (textes anonymisés) :
  - Moyenne : 0.165
  - Médiane : 0.156
  - Max : 1.000
  - Min : 0.003
Calcul du t-SNE...
Clustering DBSCAN sur l'espace TF-IDF haute dimension...
  eps=0.3: 1 clusters, 98.0% bruit
  eps=0.4: 1 clusters

## Analyse tridimentionnelle

In [10]:
#!/usr/bin/env python3
"""
Script de clustering pour l'analyse des discours politiques Tours 1+2
Basé sur le code d'analyse existant avec ajout du clustering et visualisation des clusters
"""

warnings.filterwarnings('ignore')

class ClusteringToursAnalyzer:
    def __init__(self, filepath='/Users/charlielezin/Desktop/Candidatures58-81-250825.csv'):
        self.filepath = filepath
        self.df = None
        self.similarity_matrix = None
        self.vectorizer = None
        self.embeddings_3d = None
        self.tfidf_matrix = None
        self.clusters = None
        self.n_clusters = None
        
    def load_and_filter_data(self):
        """
        Charge les données CSV et filtre pour les tours 1+2
        """
        print("Chargement des données (Tours 1+2)...")
        self.df = pd.read_csv(self.filepath, delimiter=';', encoding='utf-8')
        
        print(f"Nombre total de lignes : {len(self.df)}")
        
        # Filtrage pour les tours 1 et 2
        self.df = self.df[self.df['tour'].isin([1, 2])].copy()
        print(f"Filtrage tours 1+2 : {len(self.df)} candidatures")
        
        # Filtrer ceux qui ont des discours non vides
        self.df = self.df[
            (self.df['discours'].notna()) & 
            (self.df['discours'].str.strip() != '') &
            (self.df['discours'].str.len() > 100)
        ].copy()
        
        print(f"Nombre de candidatures avec discours : {len(self.df)}")
        self.df = self.df.reset_index(drop=True)
        
        print(f"Répartition par tour :")
        print(self.df['tour'].value_counts().sort_index())
        
        print(f"Répartition par nuance politique :")
        nuances_count = self.df['nuance_titulaire'].fillna('Inconnu').value_counts()
        print(nuances_count.head(10))
        
        return self.df
    
    def extract_names_and_parties(self):
        """
        Extrait tous les noms, prénoms et mentions de partis à supprimer
        """
        names_to_remove = set()
        parties_to_remove = set()
        
        for _, row in self.df.iterrows():
            # Noms et prénoms titulaire et suppléant
            for col in ['prenom_titulaire', 'nom_titulaire', 'prenom_suppleant', 'nom_suppleant']:
                if pd.notna(row.get(col)):
                    names_to_remove.add(str(row[col]).strip().lower())
            
            # Partis et sigles
            party_columns = [
                'parti_titulaire', 'parti_titulaire_1', 'parti_titulaire_2', 
                'parti_titulaire_3', 'parti_titulaire_4',
                'parti_suppleant', 'parti_suppleant_1', 'parti_suppleant_2',
                'parti_suppleant_3', 'parti_suppleant_4', 'parti_suppleant_5',
                'sigle_titulaire', 'sigle_suppleant', 'nuance_titulaire', 'nuance_suppleant'
            ]
            
            for col in party_columns:
                if col in row and pd.notna(row[col]):
                    party_value = str(row[col]).strip()
                    if party_value and party_value.lower() not in ['nan', 'none', '']:
                        party_parts = re.split(r'[+\-\s&/]', party_value)
                        for part in party_parts:
                            if part.strip() and len(part.strip()) > 1:
                                parties_to_remove.add(part.strip().lower())
        
        # Termes de parti communs
        common_parties = [
            'gaulliste', 'gaullistes', 'socialiste', 'socialistes', 'communiste', 'communistes',
            'radical', 'radicaux', 'centriste', 'centristes', 'républicain', 'républicains',
            'démocrate', 'démocrates', 'indépendant', 'indépendants', 'paysan', 'paysans',
            'union', 'mouvement', 'parti', 'rassemblement', 'front', 'coalition',
            'nouvelle', 'république', 'nationale', 'populaire', 'française', 'français',
            'gaulle', 'gaulles', 'gaul', 'com', 'soc', 'sfio', 'mrp', 'cnip', 'unr',
            'psu', 'agr', 'exg', 'exd', 'div', 'rdg', 'rad', 'cen'
        ]
        
        parties_to_remove.update(common_parties)
        
        # Nettoyer les sets
        names_to_remove = {name for name in names_to_remove if name and len(name) > 1}
        parties_to_remove = {party for party in parties_to_remove if party and len(party) > 2}
        
        print(f"Anonymisation:")
        print(f"   Noms/prénoms à supprimer: {len(names_to_remove)}")
        print(f"   Termes de partis à supprimer: {len(parties_to_remove)}")
        
        return names_to_remove, parties_to_remove
    
    def anonymize_text(self, text, names_to_remove, parties_to_remove):
        """
        Anonymise un texte en supprimant les noms et mentions de partis
        """
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        
        # Supprimer les noms et prénoms
        for name in names_to_remove:
            text = re.sub(rf'\b{re.escape(name)}\b', ' ', text, flags=re.IGNORECASE)
        
        # Supprimer les mentions de partis
        for party in parties_to_remove:
            text = re.sub(rf'\b{re.escape(party)}\b', ' ', text, flags=re.IGNORECASE)
        
        # Nettoyage général
        text = re.sub(r'\n+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\d+', '', text)
        text = text.strip()
        
        return text
    
    def get_french_stop_words(self):
        """
        Liste étendue de mots vides français
        """
        stop_words_base = [
            'le', 'de', 'et', 'à', 'un', 'il', 'être', 'en', 'avoir', 'que', 'pour',
            'dans', 'ce', 'son', 'une', 'sur', 'avec', 'ne', 'se', 'pas', 'tout', 'plus',
            'par', 'grand', 'comme', 'mais', 'faire', 'mettre', 'nom', 'dire', 'ces',
            'mon', 'où', 'même', 'y', 'aller', 'deux', 'moi', 'si', 'haut', 'bien', 'autre',
            'fois', 'très', 'là', 'voir', 'arriver', 'donner', 'elle', 'lui', 'tel', 'quel',
            'qui', 'du', 'des', 'aux', 'cette', 'leurs', 'nos', 'votre', 'dont', 'cette'
        ]
        
        stop_words_political = [
            'vous', 'nous', 'électeurs', 'électrices', 'france', 'français', 'française',
            'république', 'national', 'nationale', 'gouvernement', 'politique', 'pays',
            'état', 'public', 'sociale', 'social', 'économique', 'candidat', 'candidats',
            'voter', 'vote', 'élection', 'élections', 'législatives', 'député', 'assemblée',
            'monsieur', 'madame', 'citoyens', 'citoyennes', 'peuple', 'nation',
            'suppléant', 'titulaire', 'liste', 'scrutin', 'circonscription'
        ]
        
        return stop_words_base + stop_words_political
    
    def create_color_palette(self, nuances):
        """
        Palette de couleurs pour les nuances politiques
        """
        color_map = {
            'COM': '#FF0000', 'COMM': '#FF0000', 'GAUL': '#191970', 'SOC': '#FFB6C1',
            'CEN': '#87CEEB', 'EXG': '#8B0000', 'RAD': '#DAA520', 'PSU': '#FF8C00',
            'AGR': '#008000', 'CEN-D': '#0000FF', 'EXD': '#000000', 'DIV': '#808080',
            'RDG': '#FFE4E1', 'SFIO': '#FFB6C1', 'MRP': '#FF8C00', 'UDSR': '#87CEEB',
            'RGR': '#FFE4E1', 'DROITE': '#191970', 'DROIT': '#191970', 'IND': '#808080',
            'PAYSANS': '#008000'
        }
        
        # Gérer les nuances manquantes
        unique_nuances = list(set(nuances))
        remaining_nuances = [n for n in unique_nuances if n not in color_map and pd.notna(n)]
        
        if remaining_nuances:
            additional_colors = ['#DDA0DD', '#F0E68C', '#98FB98', '#F5DEB3', 
                               '#D2B48C', '#BC8F8F', '#CD853F', '#A0522D']
            for i, nuance in enumerate(remaining_nuances):
                color_map[nuance] = additional_colors[i % len(additional_colors)]
        
        color_map[np.nan] = '#CCCCCC'
        color_map['Inconnu'] = '#CCCCCC'
        color_map[None] = '#CCCCCC'
        
        return color_map
    
    def calculate_tfidf_similarity(self):
        """
        Calcule la matrice TF-IDF et la similarité sur les textes anonymisés
        """
        print("Anonymisation des textes...")
        
        # Extraire les noms et partis à supprimer
        names_to_remove, parties_to_remove = self.extract_names_and_parties()
        
        # Anonymiser tous les discours
        self.df['discours_anonymized'] = self.df['discours'].apply(
            lambda x: self.anonymize_text(x, names_to_remove, parties_to_remove)
        )
        
        # Filtrer les textes trop courts après anonymisation
        self.df = self.df[self.df['discours_anonymized'].str.len() > 50].copy()
        self.df = self.df.reset_index(drop=True)
        
        print(f"Nombre de discours après anonymisation : {len(self.df)}")
        
        if len(self.df) < 5:
            raise ValueError("Pas assez de discours après anonymisation!")
        
        print("Calcul de la matrice TF-IDF sur textes anonymisés...")
        
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            min_df=2,
            max_df=0.8,
            stop_words=self.get_french_stop_words(),
            ngram_range=(1, 2),
            lowercase=True,
            strip_accents='unicode'
        )
        
        texts = self.df['discours_anonymized'].tolist()
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)
        
        print(f"Matrice TF-IDF : {self.tfidf_matrix.shape}")
        
        print("Calcul de la similarité cosinus...")
        self.similarity_matrix = cosine_similarity(self.tfidf_matrix)
        
        # Statistiques
        similarity_upper = np.triu(self.similarity_matrix, k=1)
        non_zero_similarities = similarity_upper[similarity_upper > 0]
        
        print(f"Statistiques de similarité (textes anonymisés) :")
        print(f"  - Moyenne : {non_zero_similarities.mean():.3f}")
        print(f"  - Médiane : {np.median(non_zero_similarities):.3f}")
        print(f"  - Max : {non_zero_similarities.max():.3f}")
        print(f"  - Min : {non_zero_similarities.min():.3f}")
        
        # Calcul du t-SNE en 3D
        print("Calcul du t-SNE 3D...")
        tsne = TSNE(n_components=3, perplexity=min(30, len(self.df)-1), 
                   random_state=42, verbose=0, max_iter=1000)
        self.embeddings_3d = tsne.fit_transform(self.tfidf_matrix.toarray())
        
        return self.tfidf_matrix
    
    def find_optimal_clusters_dbscan(self):
        """Utilise DBSCAN pour identifier les clusters denses"""
        print("Clustering DBSCAN sur l'espace TF-IDF haute dimension...")
        
        # Tester différents paramètres eps pour DBSCAN
        eps_values = [0.3, 0.4, 0.5, 0.6, 0.7]
        best_eps = None
        best_n_clusters = 0
        best_noise_ratio = 1.0
        
        for eps in eps_values:
            dbscan = DBSCAN(eps=eps, min_samples=5)
            cluster_labels = dbscan.fit_predict(self.tfidf_matrix.toarray())
            
            n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
            noise_ratio = np.sum(cluster_labels == -1) / len(cluster_labels)
            
            print(f"  eps={eps}: {n_clusters} clusters, {noise_ratio:.1%} bruit")
            
            # Choisir la configuration avec un bon équilibre
            if n_clusters >= 2 and n_clusters <= 8 and noise_ratio < 0.5:
                if best_eps is None or (n_clusters > best_n_clusters and noise_ratio < best_noise_ratio):
                    best_eps = eps
                    best_n_clusters = n_clusters
                    best_noise_ratio = noise_ratio
        
        if best_eps is None:
            print("DBSCAN n'a pas trouvé de bonne configuration, utilisation de K-means avec k=4")
            return 4, "kmeans"
        
        print(f"Meilleur paramètre DBSCAN : eps={best_eps} ({best_n_clusters} clusters)")
        return best_eps, "dbscan"
    
    def perform_clustering(self, n_clusters=None):
        """Effectue le clustering sur l'espace TF-IDF haute dimension"""
        
        # Déterminer la méthode et paramètres optimaux
        param, method = self.find_optimal_clusters_dbscan()
        
        if method == "dbscan":
            print(f"Clustering DBSCAN avec eps={param}...")
            dbscan = DBSCAN(eps=param, min_samples=5)
            cluster_labels = dbscan.fit_predict(self.tfidf_matrix.toarray())
            
            # Traiter le bruit (-1) comme un cluster séparé
            self.clusters = cluster_labels
            unique_clusters = set(cluster_labels)
            self.n_clusters = len(unique_clusters)
            
        else:  # kmeans fallback
            print(f"Clustering K-means avec {param} clusters...")
            kmeans = KMeans(n_clusters=param, random_state=42, n_init=10)
            self.clusters = kmeans.fit_predict(self.tfidf_matrix.toarray())
            self.n_clusters = param
        
        # Ajouter les clusters au DataFrame
        self.df['cluster'] = self.clusters
        
        print("Répartition des clusters :")
        unique_labels = set(self.clusters)
        for label in sorted(unique_labels):
            cluster_size = np.sum(self.clusters == label)
            if label == -1:
                print(f"  Bruit: {cluster_size} documents")
            else:
                print(f"  Cluster {label}: {cluster_size} documents")
        
        return self.clusters
    
    def analyze_clusters(self):
        """Analyse détaillée des clusters"""
        print(f"ANALYSE DES CLUSTERS")
        print("="*50)
        
        unique_labels = set(self.clusters)
        for label in sorted(unique_labels):
            if label == -1:
                cluster_docs = self.df[self.df['cluster'] == -1]
                print(f"\nBRUIT ({len(cluster_docs)} documents)")
                print("-" * 30)
            else:
                cluster_docs = self.df[self.df['cluster'] == label]
                print(f"\nCLUSTER {label} ({len(cluster_docs)} documents)")
                print("-" * 30)
            
            # Répartition par nuance
            nuances = cluster_docs['nuance_titulaire'].fillna('Inconnu').value_counts()
            print(f"Nuances principales:")
            for nuance, count in nuances.head(3).items():
                pct = (count / len(cluster_docs)) * 100
                print(f"  - {nuance}: {count} ({pct:.1f}%)")
            
            # Répartition par tour
            tours = cluster_docs['tour'].value_counts()
            print(f"Tours:")
            for tour, count in tours.items():
                pct = (count / len(cluster_docs)) * 100
                print(f"  - Tour {tour}: {count} ({pct:.1f}%)")
            
            # Exemples de candidats
            print("Exemples:")
            for _, row in cluster_docs.head(3).iterrows():
                nom = f"{row['prenom_titulaire']} {row['nom_titulaire']}"
                nuance = row['nuance_titulaire'] if pd.notna(row['nuance_titulaire']) else 'Inconnu'
                tour = row.get('tour', 'N/A')
                annee = int(row['annee_scrutin']) if pd.notna(row['annee_scrutin']) else 'N/A'
                print(f"  - {nom} ({nuance}, {annee}, T{tour})")
    
    def get_cluster_hull_points_3d(self, cluster_id):
        """Calcule l'enveloppe 3D des points d'un cluster"""
        if cluster_id == -1:
            return None
            
        cluster_mask = self.clusters == cluster_id
        if np.sum(cluster_mask) < 4:  # Minimum pour une enveloppe 3D
            return None
        
        cluster_points = self.embeddings_3d[cluster_mask]
        
        try:
            from scipy.spatial import ConvexHull
            hull = ConvexHull(cluster_points)
            # Pour la 3D, on retourne les faces du hull
            return cluster_points, hull
        except:
            # Utiliser une sphère approximative si ConvexHull échoue
            center = np.mean(cluster_points, axis=0)
            distances = np.linalg.norm(cluster_points - center, axis=1)
            radius = np.max(distances) * 1.2
            return cluster_points, None
    
    def create_interactive_tsne_3d_with_clusters(self):
        """
        Visualisation t-SNE 3D interactive avec clusters entourés
        """
        print("Création de la visualisation t-SNE 3D avec clusters...")
        
        nuances = self.df['nuance_titulaire'].fillna('Inconnu')
        color_palette = self.create_color_palette(nuances)
        
        hover_texts = []
        for _, row in self.df.iterrows():
            nom = f"{row['prenom_titulaire']} {row['nom_titulaire']}"
            nuance = row['nuance_titulaire'] if pd.notna(row['nuance_titulaire']) else 'Inconnu'
            annee = int(row['annee_scrutin']) if pd.notna(row['annee_scrutin']) else 'N/A'
            tour = row.get('tour', 'N/A')
            cluster_id = row.get('cluster', 'N/A')
            departement = row.get('departement_nom', 'N/A')
            circonscription = row.get('identifiant_circonscription', 'N/A')
            
            hover_text = f"<b>{nom}</b><br><b>Nuance:</b> {nuance}<br><b>Cluster:</b> {cluster_id}<br><b>Année:</b> {annee}<br><b>Tour:</b> {tour}<br><b>Département:</b> {departement}<br><b>Circonscription:</b> {circonscription}<br><i>Analyse sur texte anonymisé</i>"
            hover_texts.append(hover_text)
        
        fig = go.Figure()
        
        # Ajouter les points par nuance politique en 3D
        for nuance in set(nuances):
            mask = nuances == nuance
            if sum(mask) > 0:
                indices = np.where(mask)[0]
                
                fig.add_trace(go.Scatter3d(
                    x=self.embeddings_3d[mask, 0],
                    y=self.embeddings_3d[mask, 1],
                    z=self.embeddings_3d[mask, 2],
                    mode='markers',
                    name=f'{nuance} (n={sum(mask)})',
                    marker=dict(
                        color=color_palette.get(nuance, '#CCCCCC'),
                        size=6,
                        line=dict(width=1, color='black'),
                        opacity=0.8
                    ),
                    hovertemplate='%{customdata}<extra></extra>',
                    customdata=[hover_texts[i] for i in indices]
                ))
        
        # Ajouter des sphères pour représenter les clusters
        cluster_colors = qualitative.Plotly[:len(set(self.clusters))]
        color_idx = 0
        
        for cluster_id in sorted(set(self.clusters)):
            if cluster_id != -1:
                cluster_mask = self.clusters == cluster_id
                if np.sum(cluster_mask) >= 4:
                    cluster_points = self.embeddings_3d[cluster_mask]
                    cluster_size = np.sum(cluster_mask)
                    
                    # Calculer le centre et le rayon du cluster
                    center = np.mean(cluster_points, axis=0)
                    distances = np.linalg.norm(cluster_points - center, axis=1)
                    radius = np.max(distances) * 1.1
                    
                    # Créer une sphère transparente pour délimiter le cluster
                    u = np.linspace(0, 2 * np.pi, 20)
                    v = np.linspace(0, np.pi, 20)
                    x_sphere = center[0] + radius * np.outer(np.cos(u), np.sin(v))
                    y_sphere = center[1] + radius * np.outer(np.sin(u), np.sin(v))
                    z_sphere = center[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
                    
                    color_hex = cluster_colors[color_idx]
                    r = int(color_hex[1:3], 16)
                    g = int(color_hex[3:5], 16)
                    b = int(color_hex[5:7], 16)
                    
                    fig.add_trace(go.Surface(
                        x=x_sphere,
                        y=y_sphere,
                        z=z_sphere,
                        opacity=0.2,
                        colorscale=[[0, f'rgba({r}, {g}, {b}, 0.2)'], [1, f'rgba({r}, {g}, {b}, 0.2)']],
                        showscale=False,
                        name=f'Cluster {cluster_id} ({cluster_size} docs)',
                        hoverinfo='name'
                    ))
                    
                color_idx += 1
        
        fig.update_layout(
            title=f'Analyse t-SNE 3D avec Clusters - Tours 1+2<br><sub>Clusters délimités par des sphères transparentes sur textes anonymisés</sub>',
            width=1400,
            height=900,
            scene=dict(
                xaxis_title='Dimension t-SNE 1',
                yaxis_title='Dimension t-SNE 2',
                zaxis_title='Dimension t-SNE 3',
                camera=dict(
                    eye=dict(x=1.2, y=1.2, z=1.2)
                )
            ),
            showlegend=True,
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.01)
        )
        
        return fig

def main():
    """
    Analyse principale avec clustering
    """
    print("ANALYSE DE CLUSTERING - DISCOURS POLITIQUES TOURS 1+2")
    print("="*60)
    
    try:
        analyzer = ClusteringToursAnalyzer()
        
        # 1. Chargement des données
        analyzer.load_and_filter_data()
        
        if len(analyzer.df) == 0:
            print("Aucun discours trouvé!")
            return
        
        # 2. Calcul TF-IDF avec anonymisation
        analyzer.calculate_tfidf_similarity()
        
        # 3. Clustering
        analyzer.perform_clustering()
        
        # 4. Analyse des clusters
        analyzer.analyze_clusters()
        
        # 5. Visualisation interactive 3D avec clusters entourés
        fig = analyzer.create_interactive_tsne_3d_with_clusters()
        fig.write_html("clustering3D.html")
        
        print(f"Visualisation 3D avec clusters sauvegardée: clustering3D.html")
        print("Analyse de clustering terminée")
        
    except Exception as e:
        print(f"Erreur : {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

ANALYSE DE CLUSTERING - DISCOURS POLITIQUES TOURS 1+2
Chargement des données (Tours 1+2)...
Nombre total de lignes : 303
Filtrage tours 1+2 : 303 candidatures
Nombre de candidatures avec discours : 303
Répartition par tour :
tour
1    217
2     86
Name: count, dtype: int64
Répartition par nuance politique :
nuance_titulaire
GAUL     62
SOC      58
COM      54
CEN      36
RAD      21
EXG      15
CEN-D    15
PSU      11
AGR      11
EXD      10
Name: count, dtype: int64
Anonymisation des textes...
Anonymisation:
   Noms/prénoms à supprimer: 370
   Termes de partis à supprimer: 135
Nombre de discours après anonymisation : 303
Calcul de la matrice TF-IDF sur textes anonymisés...
Matrice TF-IDF : (303, 1000)
Calcul de la similarité cosinus...
Statistiques de similarité (textes anonymisés) :
  - Moyenne : 0.165
  - Médiane : 0.156
  - Max : 1.000
  - Min : 0.003
Calcul du t-SNE 3D...
Clustering DBSCAN sur l'espace TF-IDF haute dimension...
  eps=0.3: 1 clusters, 98.0% bruit
  eps=0.4: 1 clust